In [ ]:


col_names = ['letter_selector_temp', 'n_records']
ls_df_list = []
for ls_nchar in range(1, 17):
    wg_df['letter_selector_temp'] = wg_df['letter_group_ranked'].str[:ls_nchar + 1]    
    ls_df = wg_df[col_names].groupby(col_names[:-1]).agg(ls_count = ('n_records', 'sum')).reset_index()    
    ls_df_list.append(ls_df

In [ ]:


tot_ls_df = pd.concat(objs=ls_df_list,axis = 0)



     


In [ ]:


tot_ls_df = tot_ls_df.drop_duplicates(subset = ['letter_selector_temp', 'ls_count'])

tot_ls_df = tot_ls_df.reset_index(drop=True)



     



     


In [ ]:


tot_ls_df['ls_index'] = tot_ls_df['letter_selector_temp'].map(get_ls_index)



     


In [ ]:

n_search_space_list = []
for ls_row_id, ls_row in tot_ls_df.iloc[:None].iterrows():    
    if ls_row_id % 1000 == 0:
        print(ls_row_id)    
    
    # get letter selector id information    
    ls_id_index = np.array(ls_row['ls_index'])    

    ##
    # BUILD A COLUMN SELECTOR
    ##
    # make sure that only values GTE 0 are selected so that the right number of
    # columns are return.
    #curr_ls_id = ls_id_index[ls_id_index >= 0]
    
    ##
    # SUBSET THE wchar_matrix by column selector
    ##    
    outcome_indices = np.all(wchar_matrix[:, ls_id_index] >= 1, axis=1)
    #print(outcome_indices.sum())
    
    # this is the sub-matrix from which to query
    #ls_wchar_matrix = wchar_matrix[outcome_indices, :]
        
    # this is the list of word group ids that correspond to the word group ids
    # in the ls_wchar_matrix
    #temp_wg_id_list = word_group_id_list[outcome_indices]
    # place into a dictionary to go from wg_id to wg_index. What is the index
    # of wg_id 675?
    # wg_id_dict = {wg_id:wg_index for wg_index, wg_id in enumerate(temp_wg_id_list)}

    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'bro'    
    #n_search_space = temp_wg_id_list.shape[0]
    n_search_space_list.append(outcome_indices.sum())

tot_ls_df['n_search_space'] = n_search_space_list

In [ ]:


tot_ls_df.to_csv(path_or_buf='search_space_count.csv', index = False)



     


In [ ]:


# try the above with cupy!
import cupy as cp



     

wchar_matrix_cp = cp.asarray(a=wchar_matrix)



     

n_search_space_list = []
for ls_row_id, ls_row in tot_ls_df.iloc[:None].iterrows():    
    if ls_row_id % 1000 == 0:
        print(ls_row_id)    
    
    # get letter selector id information    
    ls_id_index = cp.asarray(ls_row['ls_index'])    

    ##
    # BUILD A COLUMN SELECTOR
    ##
    # make sure that only values GTE 0 are selected so that the right number of
    # columns are return.
    #curr_ls_id = ls_id_index[ls_id_index >= 0]
    
    ##
    # SUBSET THE wchar_matrix by column selector
    ##    
    outcome_indices = cp.all(wchar_matrix_cp[:, ls_id_index] >= 1, axis=1)
    #print(outcome_indices.sum())
    
    # this is the sub-matrix from which to query
    #ls_wchar_matrix = wchar_matrix[outcome_indices, :]
        
    # this is the list of word group ids that correspond to the word group ids
    # in the ls_wchar_matrix
    #temp_wg_id_list = word_group_id_list[outcome_indices]
    # place into a dictionary to go from wg_id to wg_index. What is the index
    # of wg_id 675?
    # wg_id_dict = {wg_id:wg_index for wg_index, wg_id in enumerate(temp_wg_id_list)}

    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'bro'    
    #n_search_space = temp_wg_id_list.shape[0]
    n_search_space_list.append(outcome_indices.sum())



     
